In [1]:
import os
import pandas as pd
from matplotlib import pylab as plt
from numpy import mean
%pylab inline

ANSWERS = {
    'стояние' : 0,
    'ходьба' : 1,
    'бег' : 2,
    'велосипед' : 3,
    'лестница' : 4,
    'автомобиль' : 5,
    'метро' : 6,
    'электросамокат' : 7,
    'самокат' : 8
}

DATA_PATH = r'/home/chopik/Documents/minicourse/data'
KAGGLE_DATA_PATH = r'/home/chopik/Documents/minicourse/kaggle_data' 

Populating the interactive namespace from numpy and matplotlib
/home/chopik/.local/lib/python3.8/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [2]:
def ParseData(path):
    i = 0
    data = []
    target = []
    for dir_entry in sorted(os.listdir(path)):
        dir_entry_path = os.path.join(path, dir_entry)
        if os.path.isfile(dir_entry_path):
            i+=1
            with open(dir_entry_path, 'r') as my_file:
                print(dir_entry_path.split(os.path.sep)[-1])
                try:
                    df = pd.read_csv(my_file, delimiter=';')
                    if df.shape[1] == 5:
                        data.append(df)
                        target.append(dir_entry_path.split(os.path.sep)[-1])
                except UnicodeDecodeError:
                    print(dir_entry_path.split(os.path.sep)[-1])
                    pass

    for dir_entry in sorted(os.listdir(path)):
        dir_entry_path = os.path.join(path, dir_entry)
        if os.path.isfile(dir_entry_path):
            i += 1
            with open(dir_entry_path, 'r') as my_file:
                print(dir_entry_path.split(os.path.sep)[-1])
                try:
                    df = pd.read_csv(my_file, delimiter=',')
                    if df.shape[1] == 5:
                        data.append(df)
                        target.append(dir_entry_path.split(os.path.sep)[-1])
                except UnicodeDecodeError:
                    print(dir_entry_path.split(os.path.sep)[-1])
                    pass
    return data, target


In [3]:
def Action(name):
    CLASSES = {
        'тояни' : 0,
        'месте' : 0,
        'одьб' : 1,
        'аг' : 1,
        'ег' : 2,
        'елоси' : 3,
        'естн' : 4,
        'одъ' : 4,
        'втомо' : 5,
        'ашин' : 5,
        'метро' : 6,
        'лектро' : 7,
        'амок' : 8
    }
    for act in CLASSES:
        if name.lower().find(act) != -1:
            return CLASSES[act]
    return -1

In [4]:
def RotationMatrixFromVectors(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """
    a, b = (vec1 / numpy.linalg.norm(vec1)).reshape(3), (vec2 / numpy.linalg.norm(vec2)).reshape(3)
    v = numpy.cross(a, b)
    if any(v): #if not all zeros then 
        c = numpy.dot(a, b)
        s = numpy.linalg.norm(v)
        kmat = numpy.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
        return numpy.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))

    else:
        return numpy.eye(3) #cross of all zeros only occurs on identical directions

In [5]:
def Rotate(X, Y, Z, Mat):
    new_x = []
    new_y = []
    new_z = []
    
    for elem in zip(X,Y,Z):
        res = Mat.dot(elem)
        new_x.append(res[0])
        new_y.append(res[1])
        new_z.append(res[2])
        
    return new_x, new_y, new_z

In [6]:
def Integrate(lst):
    res = []
    s = 0
    for elem in lst:
        s += elem
        res.append(s)
    return res

In [7]:
import math

In [17]:
from scipy.signal import find_peaks

def ProcessData(df, comma=True, cut=True, name = "", a=-1, b=-1):
    if a == -1:
        st = 1250
    else:
        st = a
    if b == -1:
        end = -1250
    else:
        end = b
    if cut == False:
        st = 0
        end = -1
    print(name)
    if comma is True:
        df = df.applymap(lambda x: str(x).replace(',','.'))
        try:
            df['gFx'] = df['gFx'].astype(float)
            df['gFy'] = df['gFy'].astype(float)
            df['gFz'] = df['gFz'].astype(float)
        except KeyError:
            return -1
    
    yabs = None
    newFrq, newYabs = None, None
    x_, y_ = None, None
    ind = []
    try:

        Fs = 250 # частота сбора данных

        Acc = (df.gFx[st:end] ** 2 + df.gFy[st:end] ** 2 + df.gFz[st:end] ** 2) ** 0.5

        y = Acc.values
        n = len(y) # length of the signal
        k = np.arange(n)
        T = n / Fs
        frq = k / T # two sides frequency range
        frq = frq[:len(frq) // 2] # one side frequency range

        Y = np.fft.fft(y) / n # dft and normalization
        Y = Y[:n // 2]
        
        yabs = abs(Y)
        
        # fig, ax3 = plt.subplots(nrows=1, ncols=1)

        # ax3.plot(frq,yabs) # plotting the spectrum
        # ax3.set_xlim([0,30])
        # ax3.set_ylim([0,0.1])
        ind = find_peaks(yabs, height=[0.05, 1.5])[0]
    except ValueError:
        ind = []
        
    # Амплитуда
    acc = (df.gFx[st:end] ** 2 + df.gFy[st:end] ** 2 + df.gFz[st:end] ** 2) ** 0.5
    amplitude = acc.max() - acc.min()
    fur = 0 if len(ind) != 0 else 1
    print('The amplitude:', acc.max() - acc.min())
    print('The fur: ', fur)
    return amplitude, fur

In [11]:
def get_prediction(amplitude, fr):
    if amplitude < 1:
        return 0 # стояние
    if amplitude >  5:
        return 2 # бег
    if fr == 0:
        return 1 # велик
    return 3 # ходьба

In [12]:
data, target = ParseData(DATA_PATH)


3).csv
Карим Айтхаджаев - Айтхаджаев_езда_на_велосипеде(4).csv
Карим Айтхаджаев - Айтхаджаев_езда_на_велосипеде(5).csv
Карим Айтхаджаев - Айтхаджаев_поездки_в_автомобиле(2).csv
Карим Айтхаджаев - Айтхаджаев_поездки_в_автомобиле(3).csv
Карим Айтхаджаев - Айтхаджаев_поездки_в_автомобиле(4).csv
Карим Айтхаджаев - Айтхаджаев_поездки_в_автомобиле(5).csv
Карим Айтхаджаев - Айтхаджаев_стояние_на_месте(2).csv
Карим Айтхаджаев - Айтхаджаев_стояние_на_месте(3).csv
Карим Айтхаджаев - Айтхаджаев_стояние_на_месте(4).csv
Карим Айтхаджаев - Айтхаджаев_стояние_на_месте(5).csv
Карим Айтхаджаев - Айтхаджаев_ходьба(2).csv
Карим Айтхаджаев - Айтхаджаев_ходьба(3).csv
Карим Айтхаджаев - Айтхаджаев_ходьба(4).csv
Карим Айтхаджаев - Айтхаджаев_ходьба(5).csv
Кирдяшова Алиса - 01_Бег_Кирдяшова.csv
Кирдяшова Алиса - 01_Лестница_Кирдяшова.csv
Кирдяшова Алиса - 01_Стояние_Кирдяшова.csv
Кирдяшова Алиса - 01_Ходьба_Кирдяшова.csv
Кирдяшова Алиса - 02_Бег_Кирдяшова..csv
Кирдяшова Алиса - 02_Лестница_Кирдяшова.csv
Кирдя

In [18]:
amplitude = []
fure = []
for df, tr in zip(data, target):
    am, fur = math.nan, math.nan
    try:
        am, fur = ProcessData(df, name=tr)
    except TypeError:
        am, fur = math.nan, math.nan
    amplitude.append(am)
    fure.append(fur)

р - Карбаев_ходьба_4.csv
The amplitude: 2.848041521390372
The fur:  0
Карбаев Саруар - Карбаев_ходьба_5.csv
The amplitude: 3.3204185814253337
The fur:  0
Карим Айтхаджаев - Айтхаджаев_бег(2).csv
The amplitude: 2.901278762697561
The fur:  0
Карим Айтхаджаев - Айтхаджаев_бег(3).csv
The amplitude: 3.1678950268247665
The fur:  0
Карим Айтхаджаев - Айтхаджаев_бег(4).csv
The amplitude: 2.9690180329855176
The fur:  0
Карим Айтхаджаев - Айтхаджаев_бег(5).csv
The amplitude: 4.784809681717607
The fur:  0
Карим Айтхаджаев - Айтхаджаев_езда_на_велосипеде(2).csv
The amplitude: 1.991326144158299
The fur:  0
Карим Айтхаджаев - Айтхаджаев_езда_на_велосипеде(3).csv
The amplitude: 6.423952888247974
The fur:  1
Карим Айтхаджаев - Айтхаджаев_езда_на_велосипеде(4).csv
The amplitude: 4.2486847953992095
The fur:  1
Карим Айтхаджаев - Айтхаджаев_езда_на_велосипеде(5).csv
The amplitude: 1.3003862144312404
The fur:  0
Карим Айтхаджаев - Айтхаджаев_поездки_в_автомобиле(2).csv
The amplitude: 1.0126089528887894
Th

In [19]:
prediction = [Prediction(e, f) for e, f in zip(amplitude, fure)]
answer = [Action(tr) for tr in target]
result = pd.DataFrame({'track_num' : np.arange(len(data)), 'answer' : answer, 'action' : prediction})
result.head(10)

,track_num,answer,action
0,0,0,3
1,1,0,3
2,2,0,3
3,3,0,0
4,4,0,0
5,5,1,1
6,6,1,1
7,7,1,1
8,8,1,1
9,9,3,1


In [20]:
result.head(50)

,track_num,answer,action
0,0,0,3
1,1,0,3
2,2,0,3
3,3,0,0
4,4,0,0
5,5,1,1
6,6,1,1
7,7,1,1
8,8,1,1
9,9,3,1


In [24]:
from random import sample

ind0 = sample(result[result.answer == 0].index.tolist(), 32)
ind1 = sample(result[result.answer == 1].index.tolist(), 32)
ind2 = sample(result[result.answer == 2].index.tolist(), 32)
ind3 = sample(result[result.answer == 3].index.tolist(), 32)
ind4 = sample(result[result.answer == 4].index.tolist(), 32)
ind5 = sample(result[result.answer == 5].index.tolist(), 6)
ind6 = sample(result[result.answer == 6].index.tolist(), 6)
ind7 = sample(result[result.answer == 7].index.tolist(), 6)

indBalanced = ind0 + ind1 + ind2 + ind3 + ind4 + ind5 + ind6 + ind7 
len(indBalanced)

178

In [25]:
resBalanced = result.iloc[indBalanced]

In [26]:
from sklearn.metrics import accuracy_score
accuracy_score(resBalanced['answer'], resBalanced['action'])

0.5449438202247191